In [1]:
import sys
sys.path.append("../../../")
%load_ext autoreload
%autoreload 2

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from models.amp_sim import quads, grover_adaptive
from models.classical import cmaes
from models.parameters import QuadsParam, QuadsHyperParam, CMAParam, CMAHyperParam
from utils.objective_functions import get_rastrigin
import random

import time, datetime
today = datetime.datetime.fromtimestamp(time.time())
executed_time = today.strftime('%Y%m%d%H%M%S')
random.seed(42)
np.random.seed(seed=42)

パラメーター設定

In [14]:
n_dim = 3
trial_num = 100
target = np.ones(n_dim) * 0.75


init_mean = np.ones(n_dim) * 0.25
init_cov = np.identity(n_dim)
init_step_size = 0.3

rastrigin = get_rastrigin(target=target, square_term=2)
init_threshold = rastrigin(init_mean)


config = {
    "sampler_type": "quantum",
    "n_dim": n_dim,
    "n_digits": 8,
    "iter_num": 100,
    "n_samples": 4,
    "terminate_step_size": 0.001,
    "terminate_eps": 0.01,
    "optimal_amplify_num": False,
    "quantile": 0.1,
    "smoothing_th": 0.5,
    "target": target,
    "eval_limit_one_sample": 10000,
    "init_threshold": init_threshold
}



quads_param = QuadsParam(
    init_threshold, CMAParam(init_mean, init_cov, init_step_size ))


Quads

In [ ]:
eval_hists_quads = []
min_func_hists_quads = []
dist_target_hists_quads = []
for _ in tqdm(range(trial_num)):
    _, (min_func_hist, eval_num_hist, dist_target_hist, _) = quads.run_quads(rastrigin, quads_param, config, verbose=False)
    eval_hists_quads.append(eval_num_hist)
    min_func_hists_quads.append(min_func_hist)
    dist_target_hists_quads.append(dist_target_hist)

  1%|▊                                                                              | 1/100 [03:47<6:14:47, 227.14s/it]

total_eval_num:  326


  2%|█▌                                                                             | 2/100 [07:00<5:38:36, 207.32s/it]

total_eval_num:  208


  3%|██▎                                                                            | 3/100 [10:15<5:26:21, 201.88s/it]

total_eval_num:  171


In [ ]:
import pickle
with open(f"./outputs/quads_{executed_time}.pickle", mode='wb') as f:
    pickle.dump({"eval_hists": eval_hists_quads, "min_func_hists": min_func_hists_quads,
                 "dist_target": dist_target_hists_quads, "config": config}, f)

Grover

In [ ]:
min_func_hists_grover = []
eval_hists_grover = []
dist_target_hists_grover = []
for _ in tqdm(range(trial_num)):
    _, (min_func_hist, eval_num_hist, dist_target_hist, _) = grover_adaptive.run_grover_minimization(rastrigin, config, False)
    min_func_hists_grover.append(np.array(min_func_hist))
    eval_hists_grover.append(np.array(eval_hist))
    dist_target_hists_grover.append(np.array(dist_target_hist))

In [ ]:
import pickle
with open(f"./outputs/grover_{executed_time}.pickle", mode='wb') as f:
    pickle.dump({"eval_hists": eval_hists_grover, "min_fun_hists": min_func_hists_grover, 
                 "dist_target": dist_target_hists_grover, "config": config}, f)

CMAES

In [ ]:
eval_hists_cmaes = []
min_func_hists_cmaes = []
dist_target_hists_cmaes = []
cmaes_param = CMAParam(init_mean, init_cov, init_step_size ) 
for _ in tqdm(range(trial_num)):
    _, (min_func_hist, eval_num_hist, dist_target_hist, _) = cmaes.run_cmaes(rastrigin, cmaes_param, config)
    eval_hists_cmaes.append(eval_num_hist)
    min_func_hists_cmaes.append(min_func_hist)
    dist_target_hists_cmaes.append(dist_target_hist)

In [ ]:
import pickle
with open(f"./outputs/cmaes_{executed_time}.pickle", mode='wb') as f:
    pickle.dump({"eval_hists": eval_hists_cmaes, "min_fun_hists": min_func_hists_cmaes, 
                 "dist_target_hists": dist_target_hists_cmaes, "config": config}, f)